In [1]:
from openai import OpenAI

from gimkit import Query, from_openai
from gimkit import guide as g

In [ ]:
openai_client = OpenAI(api_key="", base_url="http://localhost:8000/v1")
model = from_openai(openai_client, model_name="artifacts/09251-gim-sft-tmp/sft-gim")


def print_result_and_tags(question: str):
    query = Query(question)
    result = model(query, temperature=0.0, max_tokens=8192, presence_penalty=1, seed=0)[0]
    print(result)
    print("\n" + "=" * 40 + "\n")

    # Print each tag's predicted content
    for tag in result.tags:
        print(tag)

# A 24 Puzzle QA

In [3]:
question = """4 {} 6 {} 4 {} 4 = 24""".format(*[g(desc="a single math operator") for _ in range(3)])

print_result_and_tags(question)

4 x 6 + 4 - 4 = 24


<|MASKED id="m_0" desc="a single math operator"|>x<|/MASKED|>
<|MASKED id="m_1" desc="a single math operator"|>+<|/MASKED|>
<|MASKED id="m_2" desc="a single math operator"|>-<|/MASKED|>


# A Chinese Chain-of-Thought QA

In [4]:
question = """对于淀粉和纤维素两种物质，下列说法正确的是____
A. 两者都能水解，且水解的最终产物相同
B. 两者含C、H、O三种元素的质量分数相同，且互为同分异构体
C. 它们都属于糖类，且都是溶于水的高分子化合物
D. 都可 用$\\left ( C_6H_6O_5\\right )_n$表示，但淀粉能发生银镜反应，而纤维素不能

让我们一步步思考
1. {}
2. {}
3. {}
4. {}
5. {}
6. {}
所以答案是：{}
""".format(
    *([g(desc="一个思考步骤") for _ in range(6)] + [g.options(choices=["A", "B", "C", "D"])])
)

print_result_and_tags(question)

对于淀粉和纤维素两种物质，下列说法正确的是____
A. 两者都能水解，且水解的最终产物相同
B. 两者含C、H、O三种元素的质量分数相同，且互为同分异构体
C. 它们都属于糖类，且都是溶于水的高分子化合物
D. 都可 用$\left ( C_6H_6O_5\right )_n$表示，但淀粉能发生银镜反应，而纤维素不能

让我们一步步思考
1. 首先，我们需要了解淀粉和纤维素的基本性质。淀粉是一种多糖，由葡萄糖单元组成，而纤维素也是一种多糖，但其结构与淀粉不同。两者都是由葡萄糖单元通过不同的糖苷键连接而成的高分子化合物。
2. 接下来，我们分析选项A：'两者都能水解，且水解的最终产物相同'。淀粉和纤维素都可以被水解，最终产物都是葡萄糖。因此，这个选项是正确的。
3. 然后，我们分析选项B：'两者含C、H、O三种元素的质量分数相同，且互为同分异构体'。虽然淀粉和纤维素都含有C、H、O三种元素，但由于它们的结构不同，不能称为同分异构体。因此，这个选项是错误的。
4. 接着，我们分析选项C：'它们都属于糖类，且都是溶于水的高分子化合物'。淀粉和纤维素确实都属于糖类，但纤维素不溶于水，因此这个选项是错误的。
5. 最后，我们分析选项D：'都可用$\left ( C_6H_6O_5\right )_n$表示，但淀粉能发生银镜反应，而纤维素不能'。实际上，淀粉和纤维素的化学式并不完全相同，且淀粉可以发生银镜反应是因为其含有游离的醛基，而纤维素则没有。因此，这个选项也是错误的。
6. 综上所述，只有选项A是正确的。
所以答案是：A



<|MASKED id="m_0" desc="一个思考步骤"|>首先，我们需要了解淀粉和纤维素的基本性质。淀粉是一种多糖，由葡萄糖单元组成，而纤维素也是一种多糖，但其结构与淀粉不同。两者都是由葡萄糖单元通过不同的糖苷键连接而成的高分子化合物。<|/MASKED|>
<|MASKED id="m_1" desc="一个思考步骤"|>接下来，我们分析选项A：'两者都能水解，且水解的最终产物相同'。淀粉和纤维素都可以被水解，最终产物都是葡萄糖。因此，这个选项是正确的。<|/MASKED|>
<|MASKED id="m_2" desc="一个思考步骤"|>然后，我们分析选项B：'两者含C、H、O三种元素的质量分数相同，且互为同分异构体'。虽然淀粉和纤

# Data synthesis

In [5]:
question = f"""# A short article

This small ebook is here to teach you a programming language called Forth. Forth is a language unlike most others. It’s not functional or object oriented, it doesn’t have type-checking, and it basically has zero syntax. It was written in the 70s, but is still used today for certain applications.

Why would you want to learn such an odd language? Every new programming language you learn helps you think about problems in new ways. Forth is very easy to learn, but it requires you to think in a different way than you’re used to. That makes it a perfect language to broaden your coding horizons.

This book includes a simple implementation of Forth I wrote in JavaScript. It’s by no means perfect, and is missing a lot of the functionality you’d expect in a real Forth system. It’s just here to give you an easy way to try out the examples. (If you’re a Forth expert, please contribute here and make it better!)

I’m going to assume that you know at least one other programming language, and have a basic idea of how stacks work as a data structure.

# Dependent Questions and Answers

Q: {g("A easy question about the article.")}
A: {g("A concise answer to the question.")}

---

Q: {g("A hard question about the article.")}
A: {g("A detailed answer to the question.")}

---

Q: {g("A very hard question about the article.")}
A: {g("A very detailed answer to the question.")}

# Note

- The questions should be about the article above.
- The answers should be based on the content of the article above.
- The questions are dependent, i.e., the answer to the first question may be needed to answer the second question, and so on.
"""

print_result_and_tags(question)

# A short article

This small ebook is here to teach you a programming language called Forth. Forth is a language unlike most others. It’s not functional or object oriented, it doesn’t have type-checking, and it basically has zero syntax. It was written in the 70s, but is still used today for certain applications.

Why would you want to learn such an odd language? Every new programming language you learn helps you think about problems in new ways. Forth is very easy to learn, but it requires you to think in a different way than you’re used to. That makes it a perfect language to broaden your coding horizons.

This book includes a simple implementation of Forth I wrote in JavaScript. It’s by no means perfect, and is missing a lot of the functionality you’d expect in a real Forth system. It’s just here to give you an easy way to try out the examples. (If you’re a Forth expert, please contribute here and make it better!)

I’m going to assume that you know at least one other programming la

# Controllable text generation

In [6]:
question = "On a screen, mixing #{} and #{} results in the color #{}.".format(
    g(desc="Hexadecimal color code (6 characters) of Red"),
    g(desc="Hexadecimal color code (6 characters) of Green"),
    g(desc="Hexadecimal color code (6 characters) of mixing Red and Green"),
)

print_result_and_tags(question)

On a screen, mixing #ff0000 and #00ff00 results in the color #ffff00.


<|MASKED id="m_0" desc="Hexadecimal color code (6 characters) of Red"|>ff0000<|/MASKED|>
<|MASKED id="m_1" desc="Hexadecimal color code (6 characters) of Green"|>00ff00<|/MASKED|>
<|MASKED id="m_2" desc="Hexadecimal color code (6 characters) of mixing Red and Green"|>ffff00<|/MASKED|>


# Structured text generation

In [7]:
question = f"""Complete the algorithm profile in TOML:

[algorithm_profile]
name = {g(desc="an any algorithm name")}
type = {g(desc="list[str] type; 4 items")}
performance.time_complexity: {g(desc="str type; use LaTeX expression")},
performance.space_complexity: {g(desc="str type; use LaTeX expression")}"""

print_result_and_tags(question)

Complete the algorithm profile in TOML:

[algorithm_profile]
name = Dijkstra's Algorithm
type = shortest path, graph, algorithm, single source
performance.time_complexity: O((V + E) log V),
performance.space_complexity: O(V)


<|MASKED id="m_0" desc="an any algorithm name"|>Dijkstra's Algorithm<|/MASKED|>
<|MASKED id="m_1" desc="list[str] type; 4 items"|>shortest path, graph, algorithm, single source<|/MASKED|>
<|MASKED id="m_2" desc="str type; use LaTeX expression"|>O((V + E) log V)<|/MASKED|>
<|MASKED id="m_3" desc="str type; use LaTeX expression"|>O(V)<|/MASKED|>


# Code Completion

In [8]:
# From (Fried, et al. 2022) InCoder

question = f'''def count_words(filename):
    """{g(desc="explain what the function does in one sentence")}"""
    counts = Counter()
    with open(filename) as file:
        for line in file:
            words = line.split(' ')
            counts.update(words)
    return counts
'''

print_result_and_tags(question)

def count_words(filename):
    """Count the number of words in a file and return a dictionary with the word as the key and the count as the value."""
    counts = Counter()
    with open(filename) as file:
        for line in file:
            words = line.split(' ')
            counts.update(words)
    return counts



<|MASKED id="m_0" desc="explain what the function does in one sentence"|>Count the number of words in a file and return a dictionary with the word as the key and the count as the value.<|/MASKED|>


In [9]:
# Also from (Fried, et al. 2022) InCoder

question = f'''
def {g(desc="function signature with parameters; end with a colon")}
    """ Count the number of occurrences of each word in the file. """
{g(desc="function body that implements the function")}
'''

print_result_and_tags(question)


def word_count(file_name):

    """ Count the number of occurrences of each word in the file. """
    with open(file_name, 'r') as file:
        words = file.read().split()
        word_count = {}
        for word in words:
            if word not in word_count:
                word_count[word] = 0
            word_count[word] += 1
        return word_count



<|MASKED id="m_0" desc="function signature with parameters; end with a colon"|>word_count(file_name):
<|/MASKED|>
<|MASKED id="m_1" desc="function body that implements the function"|>    with open(file_name, 'r') as file:
        words = file.read().split()
        word_count = {}
        for word in words:
            if word not in word_count:
                word_count[word] = 0
            word_count[word] += 1
        return word_count<|/MASKED|>


# Table Completion

In [10]:
question = f"""
Please fill in the missing values in the following table based on the existing values.
| Week | Milk (liters)      | Bread (loaves)     | Eggs (dozen) |
| ---- | ------------------ | ------------------ | ------------ |
| 1    | 6                  | 4                  | 2            |
| 2    | 6                  | 5                  | 3            |
| 3    | {g(desc="number")} | 4                  | 2            |
| 4    | 6                  | 5                  | 3            |
"""

print_result_and_tags(question)


Please fill in the missing values in the following table based on the existing values.
| Week | Milk (liters)      | Bread (loaves)     | Eggs (dozen) |
| ---- | ------------------ | ------------------ | ------------ |
| 1    | 6                  | 4                  | 2            |
| 2    | 6                  | 5                  | 3            |
| 3    | 6 | 4                  | 2            |
| 4    | 6                  | 5                  | 3            |



<|MASKED id="m_0" desc="number"|>6<|/MASKED|>


# KG Extraction

In [11]:
question = f"""## Content

This small ebook is here to teach you a programming language called Forth. Forth is a language unlike most others. It’s not functional or object oriented, it doesn’t have type-checking, and it basically has zero syntax. It was written in the 70s, but is still used today for certain applications.

## Extraction

Extract knowledge graph triplets (head, relation, tail) from the content.

1. ({g()}, {g()}, {g()})
2. ({g()}, {g()}, {g()})
3. ({g()}, {g()}, {g()})
4. ({g()}, {g()}, {g()})
5. ({g()}, {g()}, {g()})
6. ({g()}, {g()}, {g()})
"""

print_result_and_tags(question)

## Content

This small ebook is here to teach you a programming language called Forth. Forth is a language unlike most others. It’s not functional or object oriented, it doesn’t have type-checking, and it basically has zero syntax. It was written in the 70s, but is still used today for certain applications.

## Extraction

Extract knowledge graph triplets (head, relation, tail) from the content.

1. (Forth, is a programming language, that was written in the 70s)
2. (Forth, has zero syntax, and is still used today for certain applications)
3. (Forth, is not functional or object oriented, and doesn't have type-checking)
4. (This small ebook, is here to teach you a programming language called Forth, Forth)
5. (Forth, is unlike most others, Forth)
6. (Forth, was written in the 70s, Forth)



<|MASKED id="m_0"|>Forth<|/MASKED|>
<|MASKED id="m_1"|>is a programming language<|/MASKED|>
<|MASKED id="m_2"|>that was written in the 70s<|/MASKED|>
<|MASKED id="m_3"|>Forth<|/MASKED|>
<|MASKED id="m_